In [ ]:
#-*- coding: UTF-8 -*-
import random
from collections import defaultdict, Counter

#a tentative implementation of unsupervised HMM based on Baum-Welch algorithm
class unsupervisedHMM:
    
    def __init__(self, states, obs):
        self.initial = defaultdict(dict)
        self.transition = defaultdict(dict)
        self.emission = defaultdict(dict)
        
        self.states = states
        #M is the set of observed words
        self.M = self._counter(obs)
        self.obs = obs
        
        self.alpha = defaultdict(dict)
        self.beta = defaultdict(dict)

        self.xi = defaultdict(lambda: defaultdict(dict))
        self.gamma = defaultdict(dict)

        self.tran_prob = 0.0

    def _counter(self, nested_list):

        count = []
        for _list in nested_list:
            count += _list

        return set(count)

    #compute alpha
    def forward(self, obs):

        for state in self.states:
            self.alpha[0][state] = self.initial[state]*self.emission[state][obs[0]]

        for t in range(1, len(obs)):
            for state in self.states:
                self.alpha[t][state] = 0.0
                for state0 in self.states:
                    self.alpha[t][state] += self.alpha[t-1][state0]*self.transition[state0][state]*self.emission[state][obs[t]]

        self.tran_prob = sum(self.alpha[len(obs)-1].values())

    #compute beta
    def backward(self, obs):

        for state in self.states:
            self.beta[len(obs) - 1][state] = 1.0

        for t in range(len(obs) - 2, -1, -1):
            for state in self.states:
                self.beta[t][state] = 0.0
                for state0 in self.states:
                    self.beta[t][state] += self.transition[state][state0]*self.emission[state0][obs[t+1]]*self.beta[t+1][state0]

    #GAMMAt(i, j)= P(qt = St |O,lambda)
    #as probability in being in state i at time t and going to state j in time t+1 
    def computeGamma(self,obs):
        
        denominator = 0.0
        
        for t in range(len(obs)):
            denominator = 0.0
            for state in self.states:
                self.gamma[t][state] = self.alpha[t][state]*self.beta[t][state]
                denominator += self.gamma[t][state]
                
            for state in self.states:
                self.gamma[t][state] = self.gamma[t][state]/denominator

    #XI(i, j)= P(qt = St, qt+1 = Si+1|O,lambda)
    #as probability being in state i in time t given observation sequence and the model
    def computeXi(self, obs):

        denominator = 0.0

        for t in range(len(obs) - 1):
            denominator = 0.0
            for state in self.states:
                for state0 in self.states:
                    self.xi[t][state][state0] = self.alpha[t][state]*self.beta[t+1][state0]*self.transition[state][state0]*self.emission[state0][obs[t+1]]
                    denominator += self.xi[t][state][state0]

            for state in self.states:
                self.xi[t][state] = {key: self.xi[t][state][key]/denominator for key in self.xi[t][state].keys()}
      
    def Baum_Welch(self):

        for obr in self.obs:

            self.forward(obr)
            self.backward(obr)
            self.computeGamma(obr)
            self.computeXi(obr)

            numeratorA = 0.0
            numeratorB = 0.0
            denominatorA = 0.0
            denominatorB = 0.0

            T = len(obr)
            prev_prob = self.tran_prob
            diff = self.tran_prob + 0.001

            while self.checkConvergence(diff):
            
                for state in self.states:
                    self.initial[state] = 0.001 + (0.999*self.gamma[0][state])

                for state in self.states:
                    for state0 in self.states:
                        denominatorA = sum(self.gamma[t][state] for t in range(T-1))
                        numeratorA = sum(self.xi[t][state][state0] for t in range(T-1))
                        self.transition[state][state0] = 0.001 + (0.999*(numeratorA/denominatorA))

                    for observe in self.M:
                        denominatorB = sum(self.gamma[t][state] for t in range(T))
                        numeratorB = sum(self.gamma[t][state] for t in range(T) if observe == obr[t])
                        self.emission[state][observe] = 0.001 + (0.999*(numeratorB/denominatorB))
                        
                self.forward(obr)
                self.backward(obr)
                self.computeGamma(obr)
                self.computeXi(obr)

                diff = abs(self.tran_prob - prev_prob)
                prev_prob = self.tran_prob

    def checkConvergence(self, diff):
        if diff >= 0.001:
            return True
        return False

    def initHMM(self):

        #randomly generate the initial, transition and emission probability
        for state in self.states:
            self.initial[state] = random.random()

        for state in self.states:
            total = 0.0
            for state0 in self.states:
                self.transition[state][state0] = random.random()
                total += self.transition[state][state0]
            self.transition[state] = {key: self.transition[state][key]/total for key in self.transition[state].keys()}

        for state in self.states:
            total = 0.0
            for observe in self.M:
                self.emission[state][observe] = random.random()
                total += self.emission[state][observe]
            self.emission[state] = {key: self.emission[state][key]/total for key in self.emission[state].keys()}

    def vit_helper(self, delta):  
        max_val = 0.0
        max_key = ""  

        for key in delta.keys() :
            if delta[key] > max_val:  
                max_key = key   
                max_val = delta[key]
                
        return max_key, max_val

    def viterbi(self, obs):

        path = []  
        prob = []
        viter = {}

        #set the initial probability
        for state in self.states:
            viter[state] = self.initial[state]*self.emission[state][obs[0]]

        key, val = self.vit_helper(viter)
        path.append(key)
        prob.append(val)
        
        for t in range(1, len(obs)):
            prevState = path[-1]
            
            for state in self.states:            
                viter[state] = prob[-1]*self.transition[prevState][state]*self.emission[state][obs[t]]

            key, val = self.vit_helper(viter)
            path.append(key)  
            prob.append(val)

        print path
        return path       
        
states = ['Noun', 'Verb', 'Preposition', 'Symbol']
obs = [['He', 'saw', 'a', 'cat', '.'], ['A', 'cat', 'saw', 'him', '.'],
       ['He', 'saw', 'a', 'dog','.'], ['A','dog', 'saw', 'him','.'], ['He','chased','the','cat','.'],
       ['The','cat','chased','him','.'], ['He','chased','the','dog','.'], ['The','dog','chased','him','.']]
hmm = unsupervisedHMM(states, obs)
hmm.initHMM()
hmm.Baum_Welch()

for sent in obs:
    print sent
    hmm.viterbi(sent)
